In [4]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime
import gcsfs
xr.__version__

'0.11.0+13.g9b4a8aaa'

## Set up Dask Cluster

In [5]:
from dask.distributed import Client
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=1)
client = Client(cluster)
cluster

ImportError: cannot import name 'KubeCluster'

In [ ]:
import distributed
from time import sleep, time

def get_nworkers(cores_per_worker=2):
    cl = distributed.get_client()
    ncores = sum(cl.ncores().values())
    return ncores // cores_per_worker

def block_until_scaled(desired_workers):
    cl = distributed.get_client()
    cl.restart()
    cl.cluster.scale(desired_workers)
    while get_nworkers() != desired_workers:
        sleep(5)

In [ ]:
gc_path = 'pangeo-data/esgf_test/pr_Amon_GFDL-CM4_piControl_r1i1p1f1_gr1'
ds = xr.open_zarr(gcsfs.GCSMap(gc_path))
ds

## Benchmark Loading Speed

In [2]:
def benchmark(da, nworkers=[1, 2, 4, 8, 16], tc=np.nan):
    rows = []
    for nw in nworkers:
        block_until_scaled(nw)
        total_data_size = da.nbytes/1e6
        tic = time()
        pr_mean = da.mean(dim='time').load()
        runtime = time() - tic
        row = (datetime.now(), nw, tc, runtime, total_data_size)
        rows.append(row)
        print(', '.join([repr(r) for r in row]))
    columns = ['timestamp', 'nworkers', 'chunksize', 'runtime', 'datasize']
    df = pd.DataFrame(rows, columns=columns)
    return df

NameError: name 'np' is not defined

In [ ]:
nworkers = 
tc = 120
rows = []
for nw in nworkers:
    block_until_scaled(nw)
    total_data_size = ds.pr.nbytes/1e6
    tic = time()
    pr_mean = ds.pr.mean(dim='time').load()
    runtime = time() - tic
    row = (datetime.now(), nw, tc, runtime, total_data_size)
    rows.append(row)
    print(', '.join([repr(r) for r in row]))

In [ ]:
import pandas as pd
columns = ['timestamp', 'nworkers', 'chunksize', 'runtime', 'datasize']
df = pd.DataFrame(rows, columns=columns)
df

In [ ]:
df.to_csv('benchmark_zarr.csv')